# 트리의 앙상블
정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘 = 앙상블 학습

### Random Forest

- 부트스트랩 : 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식
- 전체 특성 중 일부 특성을 무작위로 고르고 최선의 분할을 찾는다.

RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택

RandomForestRegressor는 전체 특성 사용

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=1, random_state=42)

In [3]:
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),
      np.mean(scores['test_score'])) # 검증세트점수

0.9973541965122431 0.8905151032797809


In [4]:
rf.fit(train_input, train_target)
print(rf.feature_importances_) # 특성의 중요도
# 랜덤포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문에 중요도가 조금 상승

[0.23167441 0.50039841 0.26792718]


OOB(Out Of Bag) 샘플

부트스트랩 샘플에 포함되지않고 남는 샘플

이 샘플을 이용하여 훈련한 결정트리를 평가할 수 있다. == 검증세트의 역할

In [5]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)
# OOB 점수를 사용하면 교차 검증을 대신할 수 있어서 결과적으로 훈련세트에 더 많은 샘플을 사용할 수 있다.

0.8934000384837406


### 엑스트라 트리(Extra Trees)

- 랜덤포레스트와 동일하게 결정트리가 제공하는 대부분의 매개변수 지원
- 랜덤포레스트와 엑스트라 트리의 차이점은 부트스트랩 샘플을 사용하지 않는다는 점
- splitter = 'random'인 결정트리
- 하나의 결정트리에서 특성을 무작위로 분할한다면 성능이 낮아지겠지만 많은 트리를 앙상블하기때문에 과대적합을 막고 검증세트의 점수를 높이는 효과

In [6]:
# ExtraTreesClassifier 정의
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [7]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


### 그레이디언트부스팅(Gradient Boosting)

- 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블
- 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있다.
- 경사 하강법을 사용하여 트리 앙상블에 추가

In [8]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


그레이디언트 부스팅은 결정트리의 개수를 늘려도 과대적합에 강하다.

학습률 증가시키고 트리의 개수를 늘리면 조금 더 성능이 향상 가능

In [9]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [10]:
# 그레이디언트 부스팅도 특성 중요도를 제공
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


### 히스토그램 기반 그레이디언트 부스팅(Histogram-based Gradient Boosting)
- 정형데이터를 다루는 머신러닝 알고리즘 중 인기가 높은 알고리즘
- 입력 특성을 256개의 구간으로 나눈다.
- 256개의 구간 중 하나를 떼어 놓고 누락된 값을 위해서 사용한다 --> 입력에 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없다.


In [11]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [12]:
# 먼저 히스토그램 기반 그레이디언트 부스팅 모델을 훈련하고 훈련 세트에서 특성 중요도를 계산
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean) # 반환하는 객체는 [특성중요도, 평균, 표준편차] 를 담고있다.

[0.08876275 0.23438522 0.08027708]


In [13]:
# 테스트세트에서 특성중요도를 계산
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [14]:
# 테스트 세트에서의 성능을 최종적으로 확인
hgb.score(test_input, test_target)

0.8723076923076923

### XGBoost
- cross_validate()함수와 함께 사용가능
- tree_method 변수를 'hist'로 지정하면 히스토그램 기반 그레이디언트 부스팅 사용 가능

In [17]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/Users/yonghyunpark/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yonghyunpark/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yonghyunpark/

[04:22:22] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[04:22:23] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[04:22:23] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' t

### LightGBM

In [ ]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

In [20]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


zsh:1: command not found: nvidia-smi
